In [2]:

from scipy.io import wavfile

!pip install pydub
import wave
import contextlib
from pydub import AudioSegment

cs = "raws/cs/cs.wav"
mert = "raws/mert/mert.wav"
ortak = "raws/ortak/ortakim.wav"


x_var = '{}'.format(ortak)

sound = AudioSegment.from_wav(x_var)
sound = sound.set_channels(1)
sound.export(x_var, format="wav")


fname = x_var

with contextlib.closing(wave.open(fname,'r')) as f:
    frames = f.getnframes()
    rate = f.getframerate()
    duration = int(frames / float(rate))


def trim_wav( originalWavPath, newWavPath , start, end ):
    sampleRate, waveData = wavfile.read( originalWavPath )
    startSample = int( start * sampleRate )
    endSample = int( end * sampleRate )
    wavfile.write( newWavPath, sampleRate, waveData[startSample:endSample])
 

# fname2 = "ag.wav"
# trim_wav(fname2, fname2.replace(".wav", "_trimmed.wav"), 60,61)

for i in range(0,duration):
    trim_wav(fname, fname.replace(".wav", "_trimmed{}.wav".format(i)), i,i+1)
    print(i/duration*100)

D:\condatr\envs\wavi\lib\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


  Using cached pydub-0.25.1-py2.py3-none-any.whl (32 kB)
0.0
0.2680965147453083
0.5361930294906166
0.8042895442359249
1.0723860589812333
1.3404825737265416
1.6085790884718498
1.876675603217158
2.1447721179624666
2.4128686327077746
2.680965147453083
2.949061662198391
3.2171581769436997
3.485254691689008
3.753351206434316
4.021447721179625
4.289544235924933
4.557640750670242
4.825737265415549
5.093833780160858
5.361930294906166
5.630026809651475
5.898123324396782
6.166219839142091
6.434316353887399
6.702412868632708
6.970509383378016
7.238605898123325
7.506702412868632
7.774798927613941
8.04289544235925
8.310991957104557
8.579088471849866
8.847184986595174
9.115281501340483
9.383378016085791
9.651474530831099
9.919571045576408
10.187667560321715
10.455764075067025
10.723860589812332
10.991957104557642
11.26005361930295
11.528150134048257
11.796246648793565
12.064343163538874
12.332439678284182
12.600536193029491
12.868632707774799
13.136729222520108
13.404825737265416
13.672922252010725


In [3]:
import numpy as np
from matplotlib import pyplot as plt
import scipy.io.wavfile as wav
from numpy.lib import stride_tricks

""" short time fourier transform of audio signal """
def stft(sig, frameSize, overlapFac=0.5, window=np.hanning):
    win = window(frameSize)
    hopSize = int(frameSize - np.floor(overlapFac * frameSize))

    # zeros at beginning (thus center of 1st window should be for sample nr. 0)   
    samples = np.append(np.zeros(int(np.floor(frameSize/2.0))), sig)    
    # cols for windowing
    cols = np.ceil( (len(samples) - frameSize) / float(hopSize)) + 1
    # zeros at end (thus samples can be fully covered by frames)
    samples = np.append(samples, np.zeros(frameSize))

    frames = stride_tricks.as_strided(samples, shape=(int(cols), frameSize), strides=(samples.strides[0]*hopSize, samples.strides[0])).copy()
    frames *= win

    return np.fft.rfft(frames)    

""" scale frequency axis logarithmically """    
def logscale_spec(spec, sr=44100, factor=20.):
    timebins, freqbins = np.shape(spec)

    scale = np.linspace(0, 1, freqbins) ** factor
    scale *= (freqbins-1)/max(scale)
    scale = np.unique(np.round(scale))

    # create spectrogram with new freq bins
    newspec = np.complex128(np.zeros([timebins, len(scale)]))
    for i in range(0, len(scale)):        
        if i == len(scale)-1:
            newspec[:,i] = np.sum(spec[:,int(scale[i]):], axis=1)
        else:        
            newspec[:,i] = np.sum(spec[:,int(scale[i]):int(scale[i+1])], axis=1)

    # list center freq of bins
    allfreqs = np.abs(np.fft.fftfreq(freqbins*2, 1./sr)[:freqbins+1])
    freqs = []
    for i in range(0, len(scale)):
        if i == len(scale)-1:
            freqs += [np.mean(allfreqs[int(scale[i]):])]
        else:
            freqs += [np.mean(allfreqs[int(scale[i]):int(scale[i+1])])]

    return newspec, freqs

""" plot spectrogram"""
def plotstft(audiopath, plotpath ,binsize=2**10, colormap="jet"):
    samplerate, samples = wav.read(audiopath)

    s = stft(samples, binsize)

    sshow, freq = logscale_spec(s, factor=1.0, sr=samplerate)

    ims = 20.*np.log10(np.abs(sshow)/10e-6) # amplitude to decibel

    timebins, freqbins = np.shape(ims)


    """figsize 4.2x8,4 aspect0.1895, dpi 60"""
    plt.figure(figsize=(4.15, 8.30))
    plt.imshow(np.transpose(ims), origin="lower",aspect = "0.1897", cmap=colormap, interpolation="none")

    
    
    plt.savefig(plotpath, bbox_inches="tight",dpi = 60)

    plt.clf()

    return ims

import contextlib
from scipy.io import wavfile
import wave


import matplotlib.pyplot as plt
from scipy import signal
from scipy.io import wavfile






cs = "raws/cs/cs.wav"
mert = "raws/mert/mert.wav"
ortak = "raws/ortak/ortakim.wav"


trimmed_cs = "shorts/cs/cs_trimmed"
trimmed_mert = "shorts/mert/mert_trimmed"
trimmed_ortak = "shorts/ortak/ortakim_trimmed"


mat_cs = "matrix/cs/cs{}.png"
mat_mert = "matrix/mert/mert{}.png"
mat_ortak = "matrix/ortak/ortakim{}.png"


x_var = '{}'.format(ortak)


fname = x_var

with contextlib.closing(wave.open(fname,'r')) as f:
    frames = f.getnframes()
    rate = f.getframerate()
    duration = int(frames / float(rate))


for i in range(0,duration):
      print(i/duration*100)
      trimmed_x_var = "{}{}.wav".format(trimmed_ortak,i)
      filepath = trimmed_x_var
      plopath = mat_ortak.format(i)
      ims = plotstft(filepath,plopath)
    
# filepath = "cs_trimmed0.wav"
# plopath = "cs_trimmed1.png"
# ims = plotstft(filepath,plopath)
    
"""3 yer degistirmen gerek: x_var, trimmed_x_var, plopath """

0.0
0.2680965147453083
0.5361930294906166
0.8042895442359249
1.0723860589812333


D:\condatr\envs\wavi\lib\site-packages\ipykernel_launcher.py:58: RuntimeWarning: divide by zero encountered in log10


1.3404825737265416
1.6085790884718498
1.876675603217158
2.1447721179624666
2.4128686327077746
2.680965147453083
2.949061662198391
3.2171581769436997
3.485254691689008
3.753351206434316
4.021447721179625
4.289544235924933
4.557640750670242
4.825737265415549
5.093833780160858
5.361930294906166


D:\condatr\envs\wavi\lib\site-packages\ipykernel_launcher.py:64: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).


5.630026809651475
5.898123324396782
6.166219839142091
6.434316353887399
6.702412868632708
6.970509383378016
7.238605898123325
7.506702412868632
7.774798927613941
8.04289544235925
8.310991957104557
8.579088471849866
8.847184986595174
9.115281501340483
9.383378016085791
9.651474530831099
9.919571045576408
10.187667560321715
10.455764075067025
10.723860589812332
10.991957104557642
11.26005361930295
11.528150134048257
11.796246648793565
12.064343163538874
12.332439678284182
12.600536193029491
12.868632707774799
13.136729222520108
13.404825737265416
13.672922252010725
13.941018766756033
14.209115281501342
14.47721179624665
14.745308310991955
15.013404825737265
15.281501340482572
15.549597855227882
15.81769436997319
16.0857908847185
16.353887399463808
16.621983914209114
16.890080428954423
17.158176943699733
17.426273458445042
17.694369973190348
17.962466487935657
18.230563002680967
18.498659517426276
18.766756032171582
19.034852546916888
19.302949061662197
19.571045576407506
19.8391420911528

...

<Figure size 298.8x597.6 with 0 Axes>

<Figure size 298.8x597.6 with 0 Axes>

<Figure size 298.8x597.6 with 0 Axes>

<Figure size 298.8x597.6 with 0 Axes>

<Figure size 298.8x597.6 with 0 Axes>

In [4]:

from PIL import Image
import os, os.path

mat_cs = "matrix/cs/cs{}.png"
mat_mert = "matrix/mert/mert{}.png"
mat_ortak = "matrix/ortak/ortakim{}.png"
# mat_harry = "matrix/harry/harry{}.png"
# mat_kit = "matrix/kit/kit{}.png"
boyut = 224,224

DIR1 = 'matrix/cs'
DIR2 = 'matrix/mert'
DIR3 = "matrix/ortak"
# DIR3 = 'matrix/harry'
# DIR4 = 'matrix/kit'

l1 = len([name for name in os.listdir(DIR1) if os.path.isfile(os.path.join(DIR1, name))])
l2 = len([name for name in os.listdir(DIR2) if os.path.isfile(os.path.join(DIR2, name))])
l3 = len([name for name in os.listdir(DIR3) if os.path.isfile(os.path.join(DIR3, name))])
# l4 = len([name for name in os.listdir(DIR4) if os.path.isfile(os.path.join(DIR4, name))])

# image1 = Image.open("cs_trimmed1.png")
# new_image1 = image1.resize((boyut))
# new_image1.save("cs_resized.png")

# for i in range(0,l1):
#   image1 = Image.open(mat_cs.format(i))
#   width1, height1 = image1.size
#   print(width1, height1)
#   print(int(i+1))
#   new_image1 = image1.resize((boyut))
#   new_image1.save(mat_cs.format(i))

# print("---------------------------------------------------------------")
# for j in range(0,l2):
#   image2 = Image.open(mat_mert.format(j))
#   width2, height2 = image2.size
#   print(width2, height2)
#   print(int(j+1))
#   new_image2 = image2.resize((boyut))
#   new_image2.save(mat_mert.format(j))
  
# # print("---------------------------------------------------------------")

for k in range(0,l3):
  image3 = Image.open(mat_ortak.format(k))
  width3, height3 = image3.size
  print(width3, height3)
  print(int(k+1))
  new_image3 = image3.resize((boyut))
  new_image3.save(mat_ortak.format(k))
  
print("---------------------------------------------------------------")

# for l in range(0,l4):
#   image4 = Image.open(mat_kit.format(l))
#   width4, height4 = image4.size
#   print(width4, height4)
#   print(int(l+1))
#   new_image4 = image4.resize((boyut))
#   new_image4.save(mat_kit.format(l))

226 226
1
226 226
2
226 226
3
226 226
4
226 226
5
226 226
6
226 226
7
226 226
8
226 226
9
226 226
10
226 226
11
226 226
12
226 226
13
226 226
14
226 226
15
226 226
16
226 226
17
226 226
18
226 226
19
226 226
20
226 226
21
226 226
22
226 226
23
226 226
24
226 226
25
226 226
26
226 226
27
226 226
28
226 226
29
226 226
30
226 226
31
226 226
32
226 226
33
226 226
34
226 226
35
226 226
36
226 226
37
226 226
38
226 226
39
226 226
40
226 226
41
226 226
42
226 226
43
226 226
44
226 226
45
226 226
46
226 226
47
226 226
48
226 226
49
226 226
50
226 226
51
226 226
52
226 226
53
226 226
54
226 226
55
226 226
56
226 226
57
226 226
58
226 226
59
226 226
60
226 226
61
226 226
62
226 226
63
226 226
64
226 226
65
226 226
66
226 226
67
226 226
68
226 226
69
226 226
70
226 226
71
226 226
72
226 226
73
226 226
74
226 226
75
226 226
76
226 226
77
226 226
78
226 226
79
226 226
80
226 226
81
226 226
82
226 226
83
226 226
84
226 226
85
226 226
86
226 226
87
226 226
88
226 226
89
226 226
90
226 226
91
226 226


In [ ]:
from fastai.vision.all import *

learn = load_learner("modelim/exporti.pkl", cpu=False)

In [2]:
 !pip show fastai

Name: fastai
Version: 2.1.0
Summary: fastai simplifies training fast and accurate neural nets using modern best practices
Home-page: https://github.com/fastai/fastai/tree/master/
Author: Jeremy Howard, Sylvain Gugger, and contributors
Author-email: info@fast.ai
License: Apache Software License 2.0
Location: d:\anacondam\envs\wav\lib\site-packages
Requires: requests, fastprogress, packaging, pip, matplotlib, spacy, pillow, torchvision, scikit-learn, pandas, torch, fastcore, pyyaml, scipy
Required-by: fastbook


In [1]:
!pip show fastcore

Name: fastcore
Version: 1.3.14
Summary: Python supercharged for fastai development
Home-page: https://github.com/fastai/fastcore/tree/master/
Author: Jeremy Howard and Sylvain Gugger
Author-email: infos@fast.ai
License: Apache Software License 2.0
Location: d:\anacondam\envs\wav\lib\site-packages
Requires: pip, packaging
Required-by: nbdev, fastrelease, fastdownload, ghapi, fastai


In [4]:
!pip show torchvision

Name: torchvision
Version: 0.11.2
Summary: image and video datasets and models for torch deep learning
Home-page: https://github.com/pytorch/vision
Author: PyTorch Core Team
Author-email: soumith@pytorch.org
License: BSD
Location: d:\anacondam\envs\wav\lib\site-packages
Requires: pillow, numpy, torch
Required-by: fastai


In [5]:
!pip install tochvision==0.8.0

ERROR: Could not find a version that satisfies the requirement tochvision==0.8.0 (from versions: none)
ERROR: No matching distribution found for tochvision==0.8.0
